# Loop

Following:

http://deeplearning.net/software/theano/tutorial/loop.html

## Scan

* A general form of _recurrence_, which can be used for looping.
* _Reduction_ and _map_ (loop over the leading dimensions) are special cases of `scan`.
* We can `scan` a function along some input sequence, producing an output at each step.
* The function can see the _previous K time-steps_ of your function.
* `sum()` can be computed by scanning the `z + x(i)` function over a list, given the initial state `z=0`.
* A `for` loop may often be expressed as a `scan()` operation, and `scan` is the closest that Theano comes to looping.
* Advantages of `scan` for loops:
    * Number of iterations to be part of the symbolic graph.
    * Minimizes GPU transfers (when the GPU is involved).
    * Computes gradients through sequential steps.
    * Faster than using a `for` loop in Python with a compiled Theano function.
    * Potentially lowers overall memory usage by detecting the actual amount of memory required.
    
See also:

http://deeplearning.net/software/theano/library/scan.html#lib-scan

### Scan Example: computing `tanh(x(t).dot(W) + b` elementwise

In [1]:
cat scan_example1.py

#!/usr/bin/env python
"""
Following:
    http://deeplearning.net/software/theano/tutorial/loop.html

Scan Example: computing `tanh(x(t).dot(W) + b` elementwise
"""
from __future__ import print_function
import theano
import theano.tensor as T
import numpy as np

# define the tensor variables
X = T.matrix('X')
W = T.matrix('W')
b_sym = T.vector('b_sym')

results, updates = theano.scan(lambda v: T.tanh(T.dot(v, W) + b_sym),
                               sequences=X)
compute_elementwise = theano.function(inputs=[X, W, b_sym],
                                      outputs=[results])

# test values
x = np.eye(2, dtype=theano.config.floatX)
w = np.ones((2, 2), dtype=theano.config.floatX)
b = np.ones((2), dtype=theano.config.floatX)
b[1] = 2

print(compute_elementwise(x, w, b)[0])

# comparison with numpy
print(np.tanh(x.dot(w) + b))


In [2]:
run scan_example1.py

[[ 0.96402758  0.99505475]
 [ 0.96402758  0.99505475]]
[[ 0.96402758  0.99505475]
 [ 0.96402758  0.99505475]]


/Users/gnperdue/anaconda/envs/python2/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [3]:
print(w)

[[ 1.  1.]
 [ 1.  1.]]


In [4]:
print(x)

[[ 1.  0.]
 [ 0.  1.]]


In [5]:
print(b)

[ 1.  2.]


In [6]:
x.dot(w)

array([[ 1.,  1.],
       [ 1.,  1.]])

In [7]:
x.dot(w) + b

array([[ 2.,  3.],
       [ 2.,  3.]])

In [8]:
np.tanh(x.dot(w) + b)

array([[ 0.96402758,  0.99505475],
       [ 0.96402758,  0.99505475]])

But why `scan()` in this case? Just to show elementwise computation, I guess.

In [9]:
import theano
import theano.tensor as T
import numpy as np

X = T.matrix('X')
W = T.matrix('W')
b = T.vector('b')
results = T.tanh(X.dot(W) + b)
f = theano.function(inputs=[X,W,b], outputs=[results])
x = np.eye(2, dtype=theano.config.floatX)
w = np.ones((2, 2), dtype=theano.config.floatX)
b = np.ones((2), dtype=theano.config.floatX)
b[1] = 2

In [10]:
f(x, w, b)

[array([[ 0.96402758,  0.99505475],
        [ 0.96402758,  0.99505475]])]

### Scan Example: computing the sequence `x(t) = tanh(x(t-1).dot(W) + y(t).dot(U) + p(T - t).dot(V))`

In [11]:
cat scan_example2.py

#!/usr/bin/env python
"""
Following:
    http://deeplearning.net/software/theano/tutorial/loop.html
"""
from __future__ import print_function
import theano
import theano.tensor as T
import numpy as np

# define tensor variables
X = T.vector('X')
W = T.matrix('W')
b_sym = T.vector('b_sym')
U = T.matrix('U')
Y = T.matrix('Y')
V = T.matrix('V')
P = T.matrix('P')

results, updates = theano.scan(lambda y, p, x_tm1: T.tanh(T.dot(x_tm1, W) +
                                                          T.dot(y, U) +
                                                          T.dot(p, V)),
                               sequences=[Y, P[::-1]], outputs_info=[X])
compute_seq = theano.function(inputs=[X, W, Y, U, P, V], outputs=[results])

# test values
x = np.zeros((2), dtype=theano.config.floatX)
x[1] = 1
w = np.ones((2, 2), dtype=theano.config.floatX)
y = np.ones((5, 2), dtype=theano.config.floatX)
y[0, :] = -3
u = np.ones((2, 2), dtype=theano.config.floatX)
p = np.ones((5, 2), dtype=theano.config.f

In [12]:
run scan_example2.py

[[-0.99505475 -0.99505475]
 [ 0.96471973  0.96471973]
 [ 0.99998585  0.99998585]
 [ 0.99998771  0.99998771]
 [ 1.          1.        ]]
[[-0.99505475 -0.99505475]
 [ 0.96471973  0.96471973]
 [ 0.99998585  0.99998585]
 [ 0.99998771  0.99998771]
 [ 1.          1.        ]]
